In [1]:
import os
import pandas as pd
import numpy as np
import regex as re
from collections import defaultdict
from typing import Tuple, List, NewType
from tqdm.notebook import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import seaborn as sns 
# %config InlineBackend.figure_format = 'retina'

import Bio
from Bio.PDB import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
BioStructure = NewType('BioStructure', Bio.PDB.Structure.Structure)
BioVector = NewType('BioVector', Bio.PDB.vectors.Vector)  
BioResidue = NewType('BioResidue', Bio.PDB.Residue.Residue)  


In [3]:
#######################################################
# Load EpitopeDB
#######################################################
def load_epitopedb():
    """This function loads the Epitopes from different tables
    """
    desa = pd.read_pickle('../data/20201105_EpitopevsHLA_distance.pickle')
    return desa

#######################################################
# Load PDB file
#######################################################
def load_hla_structure(HLA_Molecule:str, path):
    parser = PDBParser()
    return parser.get_structure(HLA_Molecule, path)

#######################################################
# HLA to file name
#######################################################
def hla_to_filename(hla:str):
    """ """
    locus, specificity = hla.split('*')
    filename = '_'.join([locus, *specificity.split(':')]) + '_V1.pdb'
    return re.split('\d', locus)[0], filename
    
#######################################################
# Find HLA molecule path
#######################################################
def find_molecule_path(locus:str, filename:str) -> str:
    """This function makes use of the locus and filename resulted from 'hla_to_filename' function 
        to find the path to the relevant file .pdb file """
    
    path = os.path.expanduser(f'../data/HLAMolecule/{locus[0:2]}') # get until the first 2 character of locus if exist
    pdb_files = [file for file in os.listdir(path) if filename.split('_V1.pdb')[0] in file ]
    if len(pdb_files) != 0:
        return  True, os.path.join(path, f'{pdb_files[0]}')
    else:
        return  False, 'No path exists'

#######################################################
# Residue in short
#######################################################
def res_short(residue:BioResidue) -> str:
    """ Gets the a residue object and returns a short residue sequence_number + amino acide name code """
    
    resname = residue.get_resname()  # Residue Name
    res_code = Aminoacid_conversion.get(resname)
    res_num = residue.get_id()[1]  # Residue Number 
    return str(res_num) + res_code

Aminoacid_conversion = {
                     'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
                     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
                     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
                     'ALA': 'A', 'VAL': 'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M',
}

#######################################################
# Find the average location of Residue
#######################################################
def get_residue_avg_coord(residue:Tuple[int,str], structure:BioStructure, chain:str) -> BioVector:
    """ This function finds the average coordinate of residue by averaging all the atoms coordinates"""
    
    BioChain = structure[0][chain]
    res_num, res_code = int(residue[0]), residue[1]
    _residue = BioChain[res_num]
#     print(_residue.get_full_id())
    res_pdb = Aminoacid_conversion.get(_residue.get_resname(), 'Corresponding code of the amino-acide could not be found')
    try:
        assert res_code == res_pdb
    except AssertionError as e:
         logger.warning(f'Expected residue {res_code}, but got {res_pdb}, sequence number: {res_num}, chain: {chain},  HLA: {structure.get_id()}')
    atoms_coord = [atom.get_vector() for atom in _residue.get_atoms()]
    return np.array(atoms_coord).sum()/len(atoms_coord)

#######################################################
# Find the average location of Epitope
#######################################################
def get_epitope_avg_coord(Epitope:List[Tuple[int,str]], structure:BioStructure, HLA_chain:str) -> BioVector:
    """ This function finds the average coordinate of Epitope by averaging all the residues average coordinates"""

    residues_coord = [get_residue_avg_coord(residue, structure, HLA_chain) for residue in Epitope]
    return np.array(residues_coord).sum()/len(residues_coord)

#######################################################
# chain functions for calculating distances
#######################################################
def get_location(poly_residues:List[str], structure:BioStructure, locus:str) -> int:
    """ Locus:['A', 'B', 'C', 'DR', 'DQ'] should be max 2 letters
    """
    
    HLA_chain = {'A': 'A', 'B': 'A', 'C': 'A', 'DRB': 'B', 'DQA': 'A', 'DQB':'B'}
    epitope_coord = get_epitope_avg_coord(poly_residues, structure, HLA_chain.get(locus))
    return epitope_coord

def find_locations(EpitopeDB:pd.DataFrame) -> dict:
    hla_exceptions = ['DRB1*03:03', 'DRB1*09:02', 'A*02:06'] #'DQA1*05:01', 'DQA1*02:01','A*02:06']
    epitope_locations = defaultdict(list)
    for i in tqdm(range(0, len(EpitopeDB))): #len(EpitopeDB)
        Epitope = EpitopeDB.iloc[i].Epitope
        loc = defaultdict(list)
        for hla in EpitopeDB.iloc[i]['Luminex Alleles']:
            if hla in hla_exceptions:
                logger.warning(f'Skipped hla: {hla}')
                continue
            locus, filename = hla_to_filename(hla)
            pdb_exist, pdb_path = find_molecule_path(locus, filename)
            if pdb_exist: 
                structure = load_hla_structure(hla, pdb_path)
                poly_residues = EpitopeDB.iloc[i].PolymorphicResidues
                try: 
                    loc[hla] =  get_location(poly_residues, structure, locus).\
                                get_array().\
                                round(2).\
                                tolist()
                except KeyError as e:
                    logger.error(f'Epitope {poly_residues} HLA {structure.get_id()} "KeyError" {e}')
        epitope_locations[Epitope].append(loc)
    return epitope_locations

def write_location_df(EpitopeDB:pd.DataFrame) -> pd.DataFrame:
    epitope_locations = find_locations(EpitopeDB)
    df = (
        pd.DataFrame(epitope_locations)
        .T
        .reset_index()
        .rename(columns={'index':'Epitope', 0:'Location'})
    )
    return EpitopeDB.merge(df, on='Epitope')



In [5]:
EpitopeDB = load_epitopedb()

# Run the location script

In [6]:
# %%timeit
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
    
logger = logging.getLogger(__name__)
logging.basicConfig(filename= 'ep_distance.log',
                    filemode = 'w',
                    format= '%(name)s - %(levelname)s - %(message)s',
                    level=logging.DEBUG,
                   )


EpitopeDB_new = write_location_df(EpitopeDB)
# EpitopeDB_new = write_dsitance_df(EpitopeDB)


,Epitope,ElliPro Score,PolymorphicResidues,AntibodyReactivity,Luminex Alleles,All Alleles,distance [A],mean_distance [A],std_distance [A],mAb,Location
0,1C,High,"[(1, C)]",unknown,"{C*08:02, C*12:02, C*14:02, C*06:02, C*15:02, ...","{C*07:52, C*06:10, C*07:69, C*07:14, C*07:53, ...","{'C*06:02': 48.755, 'C*15:02': 48.759, 'C*05:0...",48.545385,0.312178,unknown,"{'C*08:02': [17.51, 19.17, 13.44], 'C*12:02': ..."
1,9D,Very Low,"[(9, D)]",unknown,"{B*08:01, C*06:02, C*18:01, C*18:02, C*07:02, ...","{C*07:52, C*06:10, C*07:69, B*08:34, C*07:14, ...","{'B*08:01': 52.644, 'C*06:02': 52.18, 'C*07:02...",52.669500,0.389487,unknown,"{'B*08:01': [3.71, -8.5, -27.76], 'C*06:02': [..."
2,9F[A],Very Low,"[(9, F)]",unknown,"{A*32:01, A*02:01, A*02:02, A*74:01, A*03:01, ...","{A*02:07, A*02:152, A*02:114, A*02:155, A*74:0...","{'A*02:03': 53.409, 'A*32:01': 52.566, 'A*80:0...",53.752900,0.530792,unknown,"{'A*32:01': [2.45, 4.33, 6.64], 'A*02:01': [20..."
3,9H,Very Low,"[(9, H)]",unknown,"{B*40:05, B*41:02, B*27:03, B*37:01, B*49:01, ...","{B*27:21, B*27:03, B*40:32, B*40:91, B*41:01, ...","{'B*40:01': 54.35, 'B*50:01': 54.077, 'B*27:05...",53.781462,0.471508,unknown,"{'B*27:03': [4.13, 103.92, -0.92], 'B*37:01': ..."
4,9S,Very Low,"[(9, S)]",unknown,"{C*14:02, A*24:03, A*30:02, A*23:01, A*23:02, ...","{A*24:22, A*24:35, A*24:96, A*01:20, A*23:15, ...","{'A*30:02': 53.668, 'A*24:03': 51.839, 'A*24:0...",51.999286,1.152629,unknown,"{'C*14:02': [17.74, -7.61, 13.22], 'A*24:03': ..."
...,...,...,...,...,...,...,...,...,...,...,...
419,160A,High,"[(160, A)]",unknown,"{DQA1*04:01, DQA1*01:02, DQA1*05:05, DQA1*06:0...","{DQA1*01:08, DQA1*05:01, DQA1*01:01, DQA1*01:0...","{'DQA1*03:01': 29.99, 'DQA1*04:01': 33.55, 'DQ...",30.801500,1.755056,unknown,"{'DQA1*04:01': [25.39, -20.61, -26.55], 'DQA1*..."
420,160AD,High,"[(160, A), (161, D)]",unknown,"{DQA1*04:01, DQA1*01:02, DQA1*06:01, DQA1*01:0...","{DQA1*01:05, DQA1*01:08, DQA1*04:01, DQA1*01:0...","{'DQA1*03:01': 31.735, 'DQA1*04:01': 35.48, 'D...",31.908000,2.145945,unknown,"{'DQA1*04:01': [26.47, -20.56, -24.86], 'DQA1*..."
421,160D,High,"[(160, D)]",unknown,"{DQA1*03:02, DQA1*03:03}","{DQA1*03:02, DQA1*03:03}","{'DQA1*03:02': 31.978, 'DQA1*03:03': 34.829}",33.403500,1.425500,unknown,"{'DQA1*03:02': [72.38, 67.98, 103.95], 'DQA1*0..."
422,160S,High,"[(160, S)]",unknown,{DQA1*05:03},"{DQA1*05:03, DQA1*05:07, DQA1*05:06}",{'DQA1*05:03': 33.099},33.099000,0.000000,unknown,"{'DQA1*05:03': [25.66, -20.38, -26.74]}"
